In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


버전 다운그레이드를 해야 하여, tensorflow를 uninstall 합니다.

In [ ]:
pip uninstall tensorflow

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? ㅛ
Your response ('ㅛ') was not one of the expected responses: y, n
Proceed (y/n)? ㅛ
Your response ('ㅛ') was not one of the expected responses: y, n
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.8.0


1.15버전부터 호환 되는 함수가 있어서, 1.15.0 버전을 불러옵니다.

In [ ]:
!pip install tensorflow==1.2.0

     |████████████████████████████████| 412.3 MB 23 kB/s 
     |████████████████████████████████| 50 kB 4.6 MB/s 
     |████████████████████████████████| 503 kB 33.1 MB/s 
     |████████████████████████████████| 3.8 MB 30.1 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=572c36c671cffcfd739f9b2caec9e43d377cd20597a45defc577e208debe69de
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succes

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.8.0'

재준님이 하셨던 것처럼, 파이썬 파일에 접근하기 위해 경로를 찾아 갑니다.

In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
cd drive/MyDrive/AIFFELTHON/AIHub감성대화_Data/AI 모델/최종모델링/NIA

/content/drive/MyDrive/AIFFELTHON/AIHub감성대화_Data/AI 모델/최종모델링/NIA


In [ ]:
ls

albert-module/                layers/                    requirements.txt
data/                         log                        save_model/
data_text/                    models/                    test_pv.sh
eval_joint_bert_allsents.py   prepare_data_mecab_bpe.py  train_joint_bert.py
eval_joint_bert_allsents.py~  __pycache__/               tutorial
eval_joint_bert.py            readers/                   utils.py
eval_joint_bert.py~           README.md                  vectorizers/


## 이제 재준님이 시도하셨던 .py 불러오기

In [ ]:
pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from models import joint_bert_fixed # models 폴더 안에, joint_bert.py 불러오기

## 이제 우리가 하려고 했던 부분을 하는 과정에서, 원래 과정은
- keras의 load함수를 이용한다. -> 그 과정에서 layer가 필요해서, alber_layer를 불러온다. 였는데, `Tensorflow 버전 1` 에서는, load함수를 불러 올 수가 없었습니다.

- 그래서 확인을 해 보니, models 폴더에 있는, joint_bert.py 내 클래스에, load 함수가 따로 있어서, 이를 이용하여 .h5 를 불러오는 방식을 채택하였습니다.

In [ ]:
# from layers import albert_layer  # layers 폴더 안에, albert_layer.py 불러오기

In [ ]:
# h5_PATH = '/content/drive/MyDrive/AIFFELTHON/AIHub감성대화_Data/AI 모델/최종모델링/NIA/save_model/epoch30/joint_bert_model.h5'

In [ ]:
# bert_hub_PATH = '/content/drive/MyDrive/AIFFELTHON/AIHub감성대화_Data/AI 모델/최종모델링/NIA/albert-module'

model을 구성하기 위해, sess 라는 파라미터가 필요한데, 이는, train_joint_bert.py 파일 즉, 학습시에 직접 지정해 준 값을 이용하여 sess를 정의하였습니다.

In [ ]:
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=0, 
                        inter_op_parallelism_threads=0,
                        allow_soft_placement=True,
                        device_count = {'GPU': 1})
sess = tf.compat.v1.Session(config=config)

In [ ]:
# model = joint_bert.JointBertModel(slots_num = 2, intents_num = 6, bert_hub_path = bert_hub_PATH, sess = sess, num_bert_fine_tune_layers=10, is_bert= False)

In [ ]:
trained_PATH = '/content/drive/MyDrive/AIFFELTHON/AIHub감성대화_Data/AI 모델/최종모델링/NIA/save_model/epoch30'

아래 함수를 그대로 이용하면, /usr/local/lib/python3.7/dist-packages/tensorflow_core/python/keras/saving/hdf5_format.py 내에, decode 관련 오류가 납니다. 이를 직접 위의 로케이션으로 접근하여, 633번줄에 `def load_weights_from_hdf5_group(f, layers):` 아래에, 645, 649 번의 .decode('utf8') 부분을 encode().decode('utf8')로 변경 한 후, 컨트롤+s를 눌러 저장을 하고 아래 코드를 돌려 오류를 수정하였습니다. (이 역시도 버전 충돌 문제라고 생각하지만 임시로 이렇게 수정하였더니 아래 코드에 오류가 나지 않았습니다.)

만일, 또 오류시, 런타임 다시시작 후 다시 돌려봅니다.

In [ ]:
load_model = joint_bert_fixed.JointBertModel.load(trained_PATH, sess)

RuntimeError: ignored

다음 과정으로는, 이제 이 모델을 이용하여, 특정 값을 input 하여 감정 분석 결과를 output 하는 함수를 아래의 코드를 분석하여 저희가 직접 짜야 하는것으로 판단하였습니다.

아래는, eval_joint_bert.py파일(테스트 구동 파일로 추정 됨) 내에서 model을 load 한 이후의 코드입니다. 아래 과정에서 어떤 값을 input 하여야하는지, 그리고 이 output을 어떻게 저희 것으로 만들어 내는지가 다음 과정이라 생각하여 아래 코드 구조를 분석 할 예정입니다.

In [ ]:
# data_text_arr, data_tags_arr, data_intents = Reader.read(data_folder_path)
data_text_arr, data_tags_arr, data_intents = Reader.read_allsents(data_folder_path)
data_input_ids, data_input_mask, data_segment_ids, data_sequence_lengths = bert_vectorizer.transform(data_text_arr)

tags_vectorizer = TagsVectorizer()
tags_vectorizer.fit(data_tags_arr)
data_tags_arr = tags_vectorizer.transform(data_tags_arr, data_input_ids)

#print(data_tags_arr[1])
#print(data_input_ids[1])

def get_results(input_ids, input_mask, segment_ids,  sequence_lengths, tags_arr,
                intents, tags_vectorizer, intents_label_encoder):
    inferred_tags, first_inferred_intent, first_inferred_intent_score, _, _, slots_score = model.predict_slots_intent([input_ids, input_mask, segment_ids], tags_vectorizer, intents_label_encoder)
    gold_tags = tags_vectorizer.simple_inverse_transform(tags_arr.astype(int), input_ids)
    #print(inferred_tags[1])
    #print(gold_tags[1])
    acc = metrics.accuracy_score(intents, first_inferred_intent)
    tag_incorrect = ''
    intent_incorrect = ''
    intent_correct = ''

    for i, sent in enumerate(input_ids):
        if intents[i] != first_inferred_intent[i]:
            tokens = tokenizer.convert_ids_to_tokens(input_ids[i])
            intent_incorrect += ('sent {}\n'.format(tokens))
            intent_incorrect += ('pred: {}\n'.format(first_inferred_intent[i].strip()))
            intent_incorrect += ('score: {}\n'.format(first_inferred_intent_score[i]))
            intent_incorrect += ('ansr: {}\n'.format(intents[i].strip()))
        else:
            tokens = tokenizer.convert_ids_to_tokens(input_ids[i])
            intent_correct += ('sent {}\n'.format(tokens))
            intent_correct += ('pred: {}\n'.format(first_inferred_intent[i].strip()))
            intent_correct += ('score: {}\n'.format(first_inferred_intent_score[i]))
            intent_correct += ('ansr: {}\n'.format(intents[i].strip()))


    # f1_score
    global positive_value
    positive_value = 0.5
    pv = positive_value
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    tp_sents = ''
    tn_sents = ''
    fp_sents = ''
    fn_sents = ''

    for i in range(len(intents)):
        if first_inferred_intent[i] == intents[i] and first_inferred_intent_score[i] >= pv:
                tp += 1
                tp_sents += ('sent {}\n'.format(tokens))
                tp_sents += ('pred: {}\n'.format(first_inferred_intent[i].strip()))
                tp_sents += ('score: {}\n'.format(first_inferred_intent_score[i]))
                tp_sents += ('ansr: {}\n'.format(intents[i].strip()))

        elif first_inferred_intent[i] != intents[i] and first_inferred_intent_score[i] >= pv:
                fp += 1
                fp_sents += ('sent {}\n'.format(tokens))
                fp_sents += ('pred: {}\n'.format(first_inferred_intent[i].strip()))
                fp_sents += ('score: {}\n'.format(first_inferred_intent_score[i]))
                fp_sents += ('ansr: {}\n'.format(intents[i].strip()))

        elif first_inferred_intent[i] == intents[i] and first_inferred_intent_score[i] < pv:
                fn += 1
                fn_sents += ('sent {}\n'.format(tokens))
                fn_sents += ('pred: {}\n'.format(first_inferred_intent[i].strip()))
                fn_sents += ('score: {}\n'.format(first_inferred_intent_score[i]))
                fn_sents += ('ansr: {}\n'.format(intents[i].strip()))

        elif first_inferred_intent[i] != intents[i] and first_inferred_intent_score[i] < pv:
                tn += 1
                tn_sents += ('sent {}\n'.format(tokens))
                tn_sents += ('pred: {}\n'.format(first_inferred_intent[i].strip()))
                tn_sents += ('score: {}\n'.format(first_inferred_intent_score[i]))
                tn_sents += ('ansr: {}\n'.format(intents[i].strip()))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = 2 * (precision * recall) / (precision + recall)
    f1_score = round(f1_score, 3)
    precision = round(precision, 3)
    recall = round(recall, 3)

    return f1_score, precision, recall, acc, intent_incorrect, intent_correct, tp, tn, fp, fn, tp_sents, tn_sents, fp_sents, fn_sents